#Preparing data for input into the hmm["activity"] models

In [2]:
from __future__ import division

%matplotlib inline
import pandas as pd
import thinkdsp
import thinkplot
import autocorr
import numpy as np

In [6]:
#Ryan's fancy way of inputting data in an easier way
data_dict = {'walking':{},'jogging':{},'upstairs':{},'downstairs':{}}
names = ['meg','ryan','dennis']
acts = ['walking', 'jogging', 'upstairs', 'downstairs']
for name in names:
    data_file_names = ['data/{}_{}_long.csv'.format(name, activity) for activity in acts]
    for i,file in enumerate(data_file_names):
        df = pd.read_csv(file)
        data_dict[acts[i]][name] = df

In [7]:
#dictionary of dictionaries where the activity is the key and the inner dictionary is each person
print data_dict

{'walking': {'meg':              x          y          z           time
0     0.493804   2.130241   8.994417  1430067490092
1     0.135272   1.395221   7.765593  1430067490265
2    -2.085350   2.178125   9.363723  1430067490445
3    -2.765303   1.742979   9.216479  1430067490625
4    -1.693299  -0.641047  10.671555  1430067490805
5    -1.965640  -1.048660   9.668983  1430067490986
6    -1.759738  -2.654571   9.588778  1430067491166
7    -6.573879  -4.705803   7.902063  1430067491346
8    -5.573701  -5.793968   1.823185  1430067491529
9    -7.328651  -5.780201  -0.530316  1430067491706
10   -6.348825  -7.636306   1.032499  1430067491886
11   -8.009203  -7.265804   0.743998  1430067492067
12   -3.308787  -3.038242 -10.119094  1430067492247
13   -7.208941  -5.977125  -0.957681  1430067492428
14   -4.558560  -6.674436  -1.050456  1430067492608
15   -5.847837  -8.037933   1.087566  1430067492787
16   -5.940613  -7.831434   0.235230  1430067492968
17   -6.208764 -13.092093  -2.680907  143006